# Deep Factorization Machines

For real-world data where inherent feature crossing structures are usually very complex and nonlinear, second-order feature interactions  generally used in factorization machines in practice are often insufficient. Modeling higher degrees of feature combinations with factorization machines is possible theoretically but it is usually not adopted due to numerical instability and high computational complexity. One effective solution is using deep neural networks. 

Deep neural networks are powerful in feature representation learning and have the potential to learn sophisticated feature interactions. As such, it is natural to integrate deep neural networks to factorization machines. Adding nonlinear transformation layers to factorization machines gives it the capability to model both low-order feature combinations and high-order feature combinations. Moreover, non-linear inherent structures from inputs can also be captured with deep neural networks. As such, we will train a representative model named deep factorization machines (DeepFM) [Guo et al., 2017] which combine FM and deep neural networks.

DeepFM consists of an FM component and a deep component which are integrated in a parallel structure. The FM component is the same as the 2-way factorization machines which is used to model the low-order feature interactions. The deep component is a multi-layered perceptron that is used to capture high-order feature interactions and nonlinearities. These two components share the same inputs/embeddings and their outputs are summed up as the final prediction. It is worth pointing out that the spirit of DeepFM resembles that of the Wide & Deep architecture which can capture both memorization and generalization. The advantages of DeepFM over the Wide & Deep model is that it reduces the effort of hand-crafted feature engineering by identifying feature combinations automatically.

![](https://drive.google.com/uc?id=1KXC_8TRNC5Dj1w_NyDfyagzxAnbQDABb)



* ntegrating neural networks to FM enables it to model complex and high-order 
interactions.

* DeepFM outperforms the original FM on the advertising dataset.


# Model implementation in PyTorch

In [ ]:
import numpy as np
import pandas as pd
import tqdm
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MovieLensDataset(Dataset):
    """
        MovieLens 1M Dataset
        Data preparation: treat samples with a rating less than 3 as negative samples
        :param dataset_path: MovieLens dataset path
    """

    def __init__(self, dataset_path, sep='::', engine='python', header=None):
        # Read the data into a Pandas dataframe
        data = pd.read_csv(dataset_path, sep=sep, engine=engine, header=header).to_numpy()[:, :3]

        # Retrieve the items and ratings data
        self.items = data[:, :2].astype(np.int) - 1  # -1 because ID begins from 1
        self.targets = self.__preprocess_target(data[:, 2]).astype(np.float32)

        # Get the range of the items
        self.field_dims = np.max(self.items, axis=0) + 1

        # Initialize NumPy arrays to store user and item indices
        self.user_field_idx = np.array((0,), dtype=np.long)
        self.item_field_idx = np.array((1,), dtype=np.long)

    def __len__(self):
        """
        :return: number of total ratings
        """
        return self.targets.shape[0]

    def __getitem__(self, index):
        """
        :param index: current index
        :return: the items and ratings at current index
        """
        return self.items[index], self.targets[index]

    def __preprocess_target(self, target):
        """
        Preprocess the ratings into negative and positive samples
        :param target: ratings
        :return: binary ratings (0 or 1)
        """
        target[target <= 3] = 0  # ratings less than or equal to 3 classified as 0
        target[target > 3] = 1  # ratings bigger than 3 classified as 1
        return target

In [ ]:
class DeepFM(nn.Module):
  """
  A Pytorch implementation of Deep Factorization Model
  """

  def __init__(self, field_dims, embed_dim, mlp_dims, dropout):
    super(DeepFM, self).__init__()
    self.linear = FeaturesLinear(field_dims)
    self.fm = FactorizationMachine(reduce_sum=True)
    self.embedding = FeaturesEmbedding(field_dims, embed_dim)
    self.embed_output_dim = len(field_dims) * embed_dim
    self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout)

  def forward(self, x):
    """
    :param x: Long tensor of size (batch_size, num_fields)
    """
    embed_x = self.embedding(x)
    x = self.linear(x) + self.fm(embed_x) + self.mlp(embed_x.view(-1, self.embed_output_dim))
    return torch.sigmoid(x.squeeze(1))

In [ ]:
class FeaturesLinear(torch.nn.Module):
    """
    Class to perform a linear transformation on the features
    """

    def __init__(self, field_dims, output_dim=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias


class FeaturesEmbedding(torch.nn.Module):
    """
    Class to get feature embeddings
    """

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)


class MultiLayerPerceptron(torch.nn.Module):
    """
    Class to instantiate a Multilayer Perceptron model
    """

    def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        super().__init__()
        layers = list()
        for embed_dim in embed_dims:
            layers.append(torch.nn.Linear(input_dim, embed_dim))
            layers.append(torch.nn.BatchNorm1d(embed_dim))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=dropout))
            input_dim = embed_dim
        if output_layer:
            layers.append(torch.nn.Linear(input_dim, 1))
        self.mlp = torch.nn.Sequential(*layers)

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        return self.mlp(x)


class FactorizationMachine(torch.nn.Module):
    """
        Class to instantiate a Factorization Machine model
    """

    def __init__(self, reduce_sum=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

In [ ]:
def fit(model, optimizer, data_loader, criterion, device, log_interval=1000):
    """
    Train the model
    :param model: choice of model
    :param optimizer: choice of optimizer
    :param data_loader: data loader class
    :param criterion: choice of loss function
    :param device: choice of device
    :return: loss being logged
    """
    # Step into train mode
    model.train()
    total_loss = 0
    for i, (fields, target) in enumerate(tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Log the total loss for every 1000 runs
        if (i + 1) % log_interval == 0:
            print('    - loss:', total_loss / log_interval)
            total_loss = 0

def test(model, data_loader, device):
    """
    Evaluate the model
    :param model: choice of model
    :param data_loader: data loader class
    :param device: choice of device
    :return: AUC score
    """
    # Step into evaluation mode
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0):
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())

    # Return AUC score between predicted ratings and actual ratings
    return roc_auc_score(targets, predicts)

In [ ]:
# get the data
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

--2021-02-09 06:12:18--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  27.2MB/s    in 0.2s    

2021-02-09 06:12:18 (27.2 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
# Get the dataset
dataset = MovieLensDataset('./ml-1m/ratings.dat')

# Split the data into 80% train, 10% validation, and 10% test
train_length = int(len(dataset) * 0.8)
valid_length = int(len(dataset) * 0.1)
test_length = len(dataset) - train_length - valid_length
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_length, valid_length, test_length))

# Instantiate data loader classes for train, validation, and test sets
train_data_loader = DataLoader(train_dataset, batch_size=1024, num_workers=8)
valid_data_loader = DataLoader(valid_dataset, batch_size=1024, num_workers=8)
test_data_loader = DataLoader(test_dataset, batch_size=1024, num_workers=8)

In [ ]:
# Get the model
field_dims = dataset.field_dims
learning_rate=0.001
weight_decay=1e-6
epoch=10
device='cpu'

model = DeepFM(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.5)
# Use binary cross entropy loss
criterion = torch.nn.BCELoss()
# Use Adam optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Loop through pre-defined number of epochs
for epoch_i in range(epoch):
    # Perform training on the train set
    fit(model, optimizer, train_data_loader, criterion, device)
    # Perform evaluation on the validation set
    valid_auc = test(model, valid_data_loader, device)
    # Log the epochs and AUC on the validation set
    print('epoch:', epoch_i, 'validation: auc:', valid_auc)

# Perform evaluation on the test set
test_auc = test(model, test_data_loader, device)
# Log the final AUC on the test set
print('test auc:', test_auc)

# Save the model checkpoint
torch.save(model.state_dict(), 'deepfm.pt')



  0%|          | 0/782 [00:00<?, ?it/s]

  5%|▍         | 36/782 [00:01<00:20, 35.65it/s]

 12%|█▏        | 97/782 [00:02<00:14, 48.24it/s]

 20%|██        | 159/782 [00:03<00:11, 52.68it/s]

 28%|██▊       | 221/782 [00:04<00:10, 52.88it/s]

 36%|███▌      | 283/782 [00:05<00:09, 54.54it/s]

 44%|████▍     | 345/782 [00:06<00:07, 55.60it/s]

 52%|█████▏    | 407/782 [00:07<00:06, 56.40it/s]

 60%|█████▉    | 469/782 [00:08<00:05, 56.97it/s]

 68%|██████▊   | 531/782 [00:09<00:04, 57.38it/s]

 76%|███████▌  | 593/782 [00:10<00:03, 57.79it/s]

 84%|████████▍ | 655/782 [00:11<00:02, 58.00it/s]

100%|██████████| 782/782 [00:13<00:00, 58.17it/s]


100%|██████████| 98/98 [00:01<00:00, 95.55it/s]


  0%|          | 0/782 [00:00<?, ?it/s]

epoch: 0 validation: auc: 0.7716942366617788




  5%|▍         | 38/782 [00:01<00:19, 37.51it/s]

 13%|█▎        | 98/782 [00:02<00:14, 48.53it/s]

 20%|██        | 158/782 [00:03<00:12, 51.96it/s]

 28%|██▊       | 218/782 [00:04<00:10, 53.88it/s]

 36%|███▌      | 278/782 [00:05<00:09, 54.97it/s]

 43%|████▎     | 339/782 [00:06<00:07, 55.95it/s]

 51%|█████     | 400/782 [00:07<00:06, 56.55it/s]

 59%|█████▉    | 461/782 [00:08<00:05, 57.00it/s]

 67%|██████▋   | 522/782 [00:09<00:04, 56.94it/s]

 75%|███████▍  | 583/782 [00:10<00:03, 56.35it/s]

 82%|████████▏ | 644/782 [00:11<00:02, 56.64it/s]

 90%|█████████ | 705/782 [00:12<00:01, 56.86it/s]

100%|██████████| 782/782 [00:13<00:00, 56.77it/s]


100%|██████████| 98/98 [00:01<00:00, 95.91it/s]


  0%|          | 0/782 [00:00<?, ?it/s]

epoch: 1 validation: auc: 0.778057202956051




  5%|▌         | 40/782 [00:01<00:18, 39.30it/s]

 13%|█▎        | 100/782 [00:02<00:13, 49.29it/s]

 21%|██        | 161/782 [00:03<00:11, 53.09it/s]

 28%|██▊       | 222/782 [00:04<00:10, 54.81it/s]

 36%|███▌      | 283/782 [00:05<00:08, 55.88it/s]

 44%|████▍     | 344/782 [00:06<00:07, 56.42it/s]

 52%|█████▏    | 405/782 [00:07<00:06, 56.90it/s]

 60%|█████▉    | 466/782 [00:08<00:05, 57.34it/s]

 67%|██████▋   | 527/782 [00:09<00:04, 57.71it/s]

 75%|███████▌  | 588/782 [00:10<00:03, 57.95it/s]

 83%|████████▎ | 649/782 [00:11<00:02, 58.21it/s]

 91%|█████████ | 710/782 [00:12<00:01, 58.40it/s]

100%|██████████| 782/782 [00:13<00:00, 58.18it/s]


100%|██████████| 98/98 [00:01<00:00, 95.07it/s]


  0%|          | 0/782 [00:00<?, ?it/s]

...

epoch: 8 validation: auc: 0.7879520626528944




  5%|▍         | 38/782 [00:01<00:19, 37.86it/s]

 13%|█▎        | 98/782 [00:02<00:14, 48.64it/s]

 20%|██        | 159/782 [00:03<00:11, 52.51it/s]

 28%|██▊       | 220/782 [00:04<00:10, 54.26it/s]

 36%|███▌      | 281/782 [00:05<00:09, 55.11it/s]

 44%|████▎     | 342/782 [00:06<00:07, 55.68it/s]

 52%|█████▏    | 403/782 [00:07<00:06, 56.08it/s]

 59%|█████▉    | 464/782 [00:08<00:05, 56.35it/s]

 67%|██████▋   | 525/782 [00:09<00:04, 56.60it/s]

 75%|███████▍  | 586/782 [00:10<00:03, 56.76it/s]

 83%|████████▎ | 647/782 [00:11<00:02, 56.95it/s]

 91%|█████████ | 708/782 [00:12<00:01, 57.19it/s]

100%|██████████| 782/782 [00:13<00:00, 56.82it/s]


100%|██████████| 98/98 [00:01<00:00, 91.17it/s]


  0%|          | 0/98 [00:00<?, ?it/s]

epoch: 9 validation: auc: 0.7886707771358512


100%|██████████| 98/98 [00:01<00:00, 93.95it/s]

test auc: 0.7920102605357593
